In [13]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.2'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 88.7 kB in 2s (40.5 kB/s)
Reading package lists... Done


In [14]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()





In [15]:
from re import T
from pyspark import SparkFiles

url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz'
spark.sparkContext.addFile(url)

music_inst_df = spark.read.csv(SparkFiles.get('amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz'), sep='\t', header=True, inferSchema=True)
music_inst_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...| 2015-08-31|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser HD203 ...|Musical Instruments| 

In [16]:

url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz'
spark.sparkContext.addFile(url)

music_df = spark.read.csv(SparkFiles.get('amazon_reviews_us_Music_v1_00.tsv.gz'), sep='\t', header=True, inferSchema=True)
music_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [18]:
rev_id = music_inst_df.select(['review_id', 'customer_id', 'product_id', 'product_parent', 'review_date'])
rev_id.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RMDCHWD0Y5OZ9|   45610553|B00HH62VB6|     618218723| 2015-08-31|
| RZSL0BALIYUNU|   14640079|B003LRN53I|     986692292| 2015-08-31|
| RIZR67JKUDBI0|    6111003|B0006VMBHI|     603261968| 2015-08-31|
|R27HL570VNL85F|    1546619|B002B55TRG|     575084461| 2015-08-31|
|R34EBU9QDWJ1GD|   12222213|B00N1YPXW2|     165236328| 2015-08-31|
|R1WCUI4Z1SIQEO|   46018513|B001N4GRGS|     134151483| 2015-08-31|
| RL5LNO26GAVJ1|   10225065|B009PJRMHQ|     694166585| 2015-08-31|
|R3GYQ5W8JHP8SB|    6356995|B00NKBDAZS|     446431775| 2015-08-31|
|R30SHYQXGG5EYC|   35297198|B006MIU7U2|     125871705| 2015-08-31|
|R14YLXA56NP51I|   32139520|B000FIBD0I|     771888534| 2015-08-31|
|R1ZH0HSH38IOTZ|   36060782|B0002E52GG|      68535945| 2015-08-31|
|R3H53KLLC210XI|    5301309|B00RZIH52G|     725541773| 2015-08

In [19]:
prod_id = music_inst_df.select(['product_id', 'product_title'])
prod_id.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00HH62VB6|AGPtek® 10 Isolat...|
|B003LRN53I|Sennheiser HD203 ...|
|B0006VMBHI|AudioQuest LP rec...|
|B002B55TRG|Hohner Inc. 560BX...|
|B00N1YPXW2|Blue Yeti USB Mic...|
|B001N4GRGS|Middle Atlantic P...|
|B009PJRMHQ|Kmise 1pc Pickgua...|
|B00NKBDAZS|Kealoha Concert U...|
|B006MIU7U2|Halco 80000 - MR1...|
|B000FIBD0I|Gator GPTBLACK Pl...|
|B0002E52GG|Hetman 1 - Light ...|
|B00RZIH52G|Dragonpad pop fil...|
|B001792BAU|DharmaObjects Rel...|
|B009GSKW1Y|Musiclily SSS Pla...|
|B0002F4TKA|Vic Firth America...|
|B00K17YFBW|Guitar Stand for ...|
|B00EQ24HJS|Generic 3PLY Faux...|
|B00IBOYTUE|Audio 2000 6525 3...|
|B00FBRUSAE|Sawtooth ST-AMP-1...|
|B0113D2QUO|Upado Unlimited G...|
+----------+--------------------+
only showing top 20 rows



In [20]:
custo_df = music_inst_df.groupby('customer_id').agg({'customer_id': 'count'})
custo_df.show()

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   27314089|                 1|
|    6711334|                 1|
|   44525866|                 4|
|   47461997|                 1|
|   42560427|                 1|
|   35225823|                 1|
|   52526865|                20|
|    1954060|                 1|
|   34202730|                 3|
|   40014361|                 1|
|   42719693|                 1|
|   27252006|                 1|
|   48297144|                 1|
|   12204397|                 1|
|   17090175|                 2|
|   13352125|                 9|
|   46958825|                28|
|     134138|                 1|
|     283456|                 1|
|   42847010|                 2|
+-----------+------------------+
only showing top 20 rows



In [23]:
vine_df = music_inst_df.select(['review_id', 'star_rating','helpful_votes', 'vine'])
vine_df.show()

+--------------+-----------+-------------+----+
|     review_id|star_rating|helpful_votes|vine|
+--------------+-----------+-------------+----+
| RMDCHWD0Y5OZ9|          3|            0|   N|
| RZSL0BALIYUNU|          5|            0|   N|
| RIZR67JKUDBI0|          3|            0|   N|
|R27HL570VNL85F|          5|            0|   N|
|R34EBU9QDWJ1GD|          5|            0|   N|
|R1WCUI4Z1SIQEO|          5|            0|   N|
| RL5LNO26GAVJ1|          2|            3|   N|
|R3GYQ5W8JHP8SB|          5|            0|   N|
|R30SHYQXGG5EYC|          5|            0|   N|
|R14YLXA56NP51I|          5|            1|   N|
|R1ZH0HSH38IOTZ|          5|            0|   N|
|R3H53KLLC210XI|          4|            0|   N|
|R3OOR877NGA8JK|          3|            0|   N|
|R1BY7WKOZ3KMH0|          2|            0|   N|
| RXP1TFSWE8EG9|          5|            0|   N|
|R3J44DPP12OTLJ|          5|            0|   N|
| RFOV69SK0T676|          5|            0|   N|
|R2HUWDNW62FOL3|          1|            

In [24]:
mode = "append"
jdbc_url="jdbc:postgresql://:5432/my_data_class_db"
config = {"user":"postgres", "password": "bootcamp", "driver":"org.postgresql.Driver"}

In [27]:
rev_id.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored